<a href="https://colab.research.google.com/github/ortenburger/apoHackathon/blob/master/PandasReadIn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os, zipfile, shutil
#os.chdir("gdrive/My Drive/Hackathon")
#for file in os.listdir("."):
#  zip_ref = zipfile.ZipFile(file, 'r')
#  zip_ref.extractall('.')
#  zip_ref.close()
#shutil.rmtree('.')


FileNotFoundError: ignored

In [0]:
!ls

2016_Qualitätsberichte.zip


In [0]:
from zipfile import ZipFile
import xml.etree.cElementTree as et

	
import xml.etree.cElementTree as et

zip_file = ZipFile('2016_Qualitätsberichte.zip')
parsedXMLs = {text_file.filename: et.parse(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.xml')}

In [0]:
for file in parsedXMLs:
  for node in parsedXMLs[file].getroot():
      icd_10 = node.attrib.get('icd_10')
      #email = node.find('email')
      #phone = node.find('phone')
      #street = node.find('address/street')

x corresponds to 1
y corresponds to 2
z corresponds to 3
